## 1. 환경 설정

`(1) Env 환경변수`

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

`(2) 기본 라이브러리`

In [2]:
import os
from glob import glob

from pprint import pprint

`(3) 벡터저장소 로드`  
- 저장해 둔 크로마 벡터저장소를 가져오기

In [3]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small", 
)

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

/Users/jeongyeong-yeon/gy_jeon/rag-master-study/myenv/lib/python3.11/site-packages/langchain_openai/chat_models/__init__.py:1: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from langchain_openai.chat_models.azure import AzureChatOpenAI
/Users/jeongyeong-yeon/gy_jeon/rag-master-study/myenv/lib/python3.11/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'allow_population_by_field_name' has been renamed to 'populate_by_name'
  warnings.warn(mess

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.10/u/custom-json-schema

In [21]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma

embeddings = GoogleGenerativeAIEmbeddings(model='models/text-embedding-004', google_api_key=os.environ["GEMINI_API_KEY"])

vectorstore = Chroma(
    embedding_function=embeddings,
    collection_name="chroma_test",
    persist_directory="./chroma_db",
    )

print(f"벡터 저장소에 저장된 문서 수: {vectorstore._collection.count()}")

벡터 저장소에 저장된 문서 수: 10


## 2. LangChain LCEL

### 2.1 Prompt + LLM

In [4]:
# 다중 메시지 전송
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

# 모델 초기화
llm = ChatOpenAI(
    model="gpt-4o-mini", 
    temperature=0.3, 
    max_tokens=100,
    )

messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}"),
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

# 템플릿을 출력
print(prompt)

PydanticUserError: The `__modify_schema__` method is not supported in Pydantic v2. Use `__get_pydantic_json_schema__` instead in class `SecretStr`.

For further information visit https://errors.pydantic.dev/2.10/u/custom-json-schema

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
# 다중 메시지 전송
from langchain_core.prompts import ChatPromptTemplate

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.3,
    max_tokens=100,
    google_api_key=os.environ["GEMINI_API_KEY"]
)

messages = [
    ("system", "You are a helpful assistant."),
    ("user", "{query}"),
]

# 메시지 리스트를 템플릿으로 변환
prompt = ChatPromptTemplate.from_messages(messages)

# 템플릿을 출력
print(prompt)

input_variables=['query'] input_types={} partial_variables={} messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]


In [7]:
# 템플릿 입력 변수를 출력
print(prompt.input_variables)

['query']


In [8]:
# input 값을 전달하여 프롬프트를 렌더링
prompt_text = prompt.format(query="테슬라 창업자는 누구인가요?")

print(prompt_text)

System: You are a helpful assistant.
Human: 테슬라 창업자는 누구인가요?


In [9]:
# 모델에 prompt text를 직접 입력
response = llm.invoke(prompt_text)

# 모델의 응답을 출력
print(response.content)

테슬라 창업자는 일론 머스크입니다.  하지만 테슬라의 초기 창업에는 마틴 에버하르트와 마크 타펜닝도 중요한 역할을 했습니다.


In [10]:
# LCEL 체인을 구성
chain = prompt | llm

# 체인을 출력
print(chain)

first=ChatPromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), additional_kwargs={})]) middle=[] last=ChatGoogleGenerativeAI(model='models/gemini-1.5-flash', google_api_key=SecretStr('**********'), temperature=0.3, max_output_tokens=100, client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x16ccb93d0>, default_metadata=())


In [11]:
# 체인의 입력 스키마를 출력
from pprint import pprint
pprint(chain.input_schema.schema())

{'properties': {'query': {'title': 'Query', 'type': 'string'}},
 'required': ['query'],
 'title': 'PromptInput',
 'type': 'object'}


/var/folders/jp/p_d2kmdn4sxcmg9fw29g0gzh0000gn/T/ipykernel_47399/995298498.py:3: PydanticDeprecatedSince20: The `schema` method is deprecated; use `model_json_schema` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  pprint(chain.input_schema.schema())


In [12]:
# 체인을 실행 - 옵션 1
response = chain.invoke({"query":"테슬라 창업자는 누구인가요?"})

# 체인의 응답을 출력
print(response.content)

테슬라 창업자는 일론 머스크입니다.  하지만 테슬라를 공동 창업한 사람들이 더 있습니다.  일론 머스크는 초기 투자자이자 현재 CEO이지만,  Martin Eberhard, Marc Tarpenning, Ian Wright, JB Straubel도 공동 창업자로 인정받습니다.  하지만 대중적으로 가장 잘 알려진 창업자는 일론 머스크입니다.


In [13]:
# 체인을 실행 - 옵션 2
response = chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
print(response.content)

테슬라 창업자는 일론 머스크입니다.  하지만 테슬라를 공동 창업한 사람들이 더 있습니다.  일론 머스크는 초기 투자자이자 현재 CEO이지만,  Martin Eberhard, Marc Tarpenning, Ian Wright, JB Straubel도 공동 창업자로 인정받습니다.


### 2.2 Prompt + LLM + Output Parser

`a) 문자열 파싱 - StrOutputParser`

In [14]:
response

AIMessage(content='테슬라 창업자는 일론 머스크입니다.  하지만 테슬라를 공동 창업한 사람들이 더 있습니다.  일론 머스크는 초기 투자자이자 현재 CEO이지만,  Martin Eberhard, Marc Tarpenning, Ian Wright, JB Straubel도 공동 창업자로 인정받습니다.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []}, id='run-97f0e5e0-946d-4040-b12f-b3adc3fd7847-0', usage_metadata={'input_tokens': 20, 'output_tokens': 75, 'total_tokens': 95, 'input_token_details': {'cache_read': 0}})

In [15]:
# StrOutputParser - 문자열 출력을 파싱
from langchain_core.output_parsers import StrOutputParser

# 출력 파서를 생성
output_parser = StrOutputParser()

# 출력 파서를 실행
output_parser.invoke(response)

'테슬라 창업자는 일론 머스크입니다.  하지만 테슬라를 공동 창업한 사람들이 더 있습니다.  일론 머스크는 초기 투자자이자 현재 CEO이지만,  Martin Eberhard, Marc Tarpenning, Ian Wright, JB Straubel도 공동 창업자로 인정받습니다.'

In [16]:
str_chain = prompt | llm  | output_parser

query = "리비안의 설립년도는 언제인가요?"
str_response = str_chain.invoke(query)

print(str_response)



리비안은 **2009년**에 설립되었습니다.


`b) JSON 출력 - JsonOutputParser`

In [23]:
from langchain_core.output_parsers import JsonOutputParser

# 출력 파서를 생성
json_parser = JsonOutputParser()

# 체인을 실행 (JSON 출력)
json_response = chain.invoke("테슬라 창업자는 누구인가요? JSON 형식으로 출력해주세요.") 
print(json_response)

# 출력 파서를 실행
json_parser_output = json_parser.invoke(json_response)
print(json_parser_output)

content='```json\n{\n  "founder": "Elon Musk"\n}\n```' additional_kwargs={} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'safety_ratings': []} id='run-b27a6087-3de4-43b1-a920-25c8822261fc-0' usage_metadata={'input_tokens': 29, 'output_tokens': 18, 'total_tokens': 47, 'input_token_details': {'cache_read': 0}}
{'founder': 'Elon Musk'}


`c) Schema 지정 - PydanticOutputParser`

In [31]:
from langchain_core.output_parsers import PydanticOutputParser
# from langchain_core.pydantic_v1 import BaseModel, Field, validator
from pydantic import BaseModel, Field

# Pydantic 모델을 생성
class Person(BaseModel):
    """Information about a person."""

    name: str = Field(..., description="The name of the person")
    title: str = Field(..., description="The title or position of the person.")

# 출력 파서를 생성
person_parser = PydanticOutputParser(pydantic_object=Person)
print("========================================")
print("PydanticOutputParser 프롬프트")
print("----------------------------------------")
print(person_parser.get_format_instructions())
print("========================================")


# Prompt 템플릿을 생성 - Pydantic 모델을 사용
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer the user query. Wrap the output in `json` tags\n{format_instructions}",
        ),
        ("human", "{query}"),
    ]
).partial(format_instructions=person_parser.get_format_instructions())

print("Prompt 템플릿")
print("----------------------------------------")
print(prompt.format(query="테슬라 창업자는 누구인가요?"))
print("========================================")


PydanticOutputParser 프롬프트
----------------------------------------
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"name": {"description": "The name of the person", "title": "Name", "type": "string"}, "title": {"description": "The title or position of the person.", "title": "Title", "type": "string"}}, "required": ["name", "title"]}
```
Prompt 템플릿
----------------------------------------
System: Answer the user query. Wrap the output in `json` tags
The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"prope

In [19]:
# 체인을 구성
person_chain = prompt | llm | person_parser

# 체인을 실행
response = person_chain.invoke("테슬라 창업자는 누구인가요?")

# 체인의 응답을 출력
response

Person(name='엘론 머스크', title='CEO 및 창립자')

## 3. Chat Completion Methods

`(1) stream`  
- 입력에 대한 응답을 실시간 스트림을 생성하여 전달

In [27]:
import time 

for chunk in llm.stream("테슬라 창업자는 누구인가요?"):
    # 기본적으로 print 함수는 출력을 할 때마다 줄바꿈을 하지만, 줄바꿈 없이 출력하려면 end=""를 사용하면 됩니다.
    # flush=True 옵션을 사용하여 출력 버퍼를 즉시 비웁니다. 데이터를 지연 없이 즉시 출력하는 데 유용합니다.
    print(chunk.content, end="", flush=True)  
    # time.sleep(0.1)  # 0.1초 대기 (100ms)

테슬라 창업자는 **일론 머스크**입니다.  물론, 초기 단계에는 다른 공동 창업자들이 있었지만, 현재 테슬라의 비전과 방향을 가장 크게 이끌고 있는 인물은 일론 머스크입니다.

`(2) batch`  
- 입력 리스트에 대한 응답을 배치 단위로 생성

In [28]:
questions = [
    "테슬라의 창업자는 누구인가요?",
    "리비안의 창업자는 누구인가요?",
]

responses = llm.batch(questions)

for response in responses:
    response.pretty_print()
    print()

================================== Ai Message ==================================

테슬라의 창업자는 일론 머스크입니다.  하지만 테슬라 모터스는 일론 머스크가 단독으로 창업한 것이 아니라, 마틴 에버하르트와 마크 타펜닝이 2003년에 설립한 회사에 일론 머스크가 투자하고 이사회에 합류하면서 현재의 모습을 갖추게 되었습니다.

================================== Ai Message ==================================

리비안의 창업자는 **R.J. 스캐린지(R.J. Scaringe)**입니다.



## 4. Runnable

`(1) RunnableParallel`

In [23]:
# 문서 검색기 생성
retriever = vectorstore.as_retriever(
    search_kwargs={'k': 1}, 
)

query = "테슬라 창업자는 누구인가요?"
retrieved_docs = retriever.invoke(query)

retrieved_docs_text = "\n".join([doc.page_content for doc in retrieved_docs])

pprint(retrieved_docs_text)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.')


In [24]:
from langchain_core.runnables import RunnableParallel
from operator import itemgetter

# RunnableParrellel 구성
runnable = RunnableParallel(
    {"context": itemgetter("context") , "question": itemgetter("question")}
)

# 객체를 실행
response = runnable.invoke({"context": retrieved_docs_text, "question": query})

# 응답을 출력
response

{'context': '테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. 머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.',
 'question': '테슬라 창업자는 누구인가요?'}

`(2) RunnablePassthrough`

In [26]:
from langchain_core.runnables import RunnablePassthrough

runnable = RunnableParallel(
    question=RunnablePassthrough(),
)

runnable.invoke({"query":"테슬라 창업자는 누구인가요?"})

{'question': {'query': '테슬라 창업자는 누구인가요?'}}

`(3) RunnableLambda`
- 정의: 파이썬의 커스텀 함수를 매핑하는데 사용

In [27]:
from langchain_core.runnables import RunnableLambda

def count_num_words(text):
    return len(text.split())

runnable = RunnableParallel(
    question=RunnablePassthrough(),
    word_count=RunnableLambda(count_num_words),
)

runnable.invoke("테슬라 창업자는 누구인가요?") 

{'question': '테슬라 창업자는 누구인가요?', 'word_count': 3}

## 5. 전체 RAG 파이프라인 구성

`(1) RAG 프롬프트 템플릿`  

In [28]:
# Prompt 템플릿을 생성
from langchain.prompts import ChatPromptTemplate

template = """Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]
"""

prompt = ChatPromptTemplate.from_template(template)

# 템플릿을 출력
prompt.pretty_print()

================================ Human Message =================================

Answer the question based only on the following context.
Do not use any external information or knowledge. 
If the answer is not in the context, answer "잘 모르겠습니다.".

[Context]
{context}

[Question] 
{question}

[Answer]



`(2) Retriever Chain 연결`  

In [29]:
# 벡터 검색기
retriever = vectorstore.as_retriever(search_kwargs={'k': 2})

# 문서 포맷터 함수
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

# 체인 구성
retriever_chain = retriever | format_docs

# 체인을 실행
response = retriever_chain.invoke("테슬라 창업자는 누구인가요?")

pprint(response)

('테슬라(Tesla, Inc.)는 텍사스주 오스틴에 본사를 둔 미국의 대표적인 전기차 제조업체입니다. 2003년 마틴 에버하드(CEO)와 '
 '마크 타페닝(CFO)에 의해 설립된 테슬라는 2004년 페이팔과 Zip2의 공동 창업자인 일론 머스크의 참여로 큰 전환점을 맞았습니다. '
 '머스크는 최대 주주이자 회장으로서 회사를 현재의 성공으로 이끌었습니다. 회사 이름은 유명한 물리학자이자 전기공학자인 니콜라 테슬라의 '
 '이름을 따서 지어졌습니다. 테슬라는 2010년 6월 나스닥에 상장되었습니다.\n'
 '\n'
 '2023년 테슬라는 1,808,581대의 차량을 판매하여 2022년에 비해 37.65% 증가했습니다. 2012년부터 2023년 3분기까지 '
 '테슬라의 전 세계 누적 판매량은 4,962,975대를 초과했습니다. SMT Packaging에 따르면, 2023년 테슬라의 판매량은 전 '
 '세계 전기차 시장의 약 12.9%를 차지했습니다.')


`(3) RAG Chain 연결`  

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI


# LLM 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0, max_tokens=100)

# 체인 생성
rag_chain = (
    {"context": retriever_chain , "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# 체인 실행
query = "테슬라 창업자는 누구인가요?"
response = rag_chain.invoke(query)

In [31]:
# 결과 출력
response

'마틴 에버하드와 마크 타페닝입니다.'

## 6. Gradio 챗봇

`(1) invoke 실행` 

In [32]:
import gradio as gr

def answer_invoke(message, history):
    response = rag_chain.invoke(message)
    return response

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

/Users/steve2/Library/Caches/pypoetry/virtualenvs/langchain-env-9sqiXrer-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [33]:
# Graiio 종료
demo.close()

Closing server running on port: 7860


`(2) stream 실행` 

In [34]:
import gradio as gr

def answer_invoke(message, history):
    partial_message = ""
    for chunk in rag_chain.stream(message):
        if chunk is not None:
            partial_message = partial_message + chunk
            time.sleep(0.1)
            yield partial_message

# Graiio 인터페이스 생성 
demo = gr.ChatInterface(fn=answer_invoke, title="QA Bot")

# Graiio 실행  
demo.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


In [35]:
# Graiio 종료
demo.close()

Closing server running on port: 7860
